In [1]:
import pandas as pd

In [3]:
binary = pd.read_csv('/opt/ml/recent/output_binary.csv')
multiple = pd.read_csv('/opt/ml/recent/output.csv')

In [4]:
all_data = pd.concat([binary, multiple], axis = 1)
all_data

,id,binary_label,logits,id,pred_label,probs
0,0,1,[5.9140625],0,org:product,"[0.0028660793, 0.12103854, 0.85007775, 0.00054..."
1,1,1,[4.36328125],1,per:alternate_names,"[0.0001525762, 0.00024262846, 0.00033883585, 0..."
2,2,0,[-6.16015625],2,org:dissolved,"[0.00025318208, 0.00016274984, 0.00014369638, ..."
3,3,0,[-0.1177978515625],3,per:colleagues,"[0.0018266583, 0.0007449155, 0.00027849054, 0...."
4,4,0,[-0.1751708984375],4,org:top_members/employees,"[0.9991072, 4.880631e-05, 2.0395228e-05, 1.794..."
...,...,...,...,...,...,...
7760,7760,0,[-0.98779296875],7760,org:place_of_headquarters,"[0.00089434313, 0.0015392619, 0.00023413071, 4..."
7761,7761,0,[-4.109375],7761,org:product,"[0.0050781886, 0.03522381, 0.359239, 0.0046045..."
7762,7762,1,[8.0703125],7762,org:top_members/employees,"[0.99944526, 2.1029215e-05, 1.3179038e-05, 1.7..."
7763,7763,1,[4.76171875],7763,per:date_of_birth,"[0.0004302161, 0.00025689378, 0.000724003, 0.0..."


In [5]:
new_label = []
new_probs = []

no_probs = [0]*30
no_probs[0] = 1

In [8]:
for i in range(len(all_data)):
    if all_data.iloc[i]['binary_label'] < 0.5:
        new_label.append('no_relation')
        no_probs = [1-all_data.iloc[i]['binary_label']]
        tmp = [round((all_data.iloc[i]['binary_label'])/29, 12)] * 29
        no_probs.extend(tmp)
        new_probs.append(no_probs)
    else:
        new_label.append(all_data.iloc[i]['pred_label'])
        tmp = eval(all_data.iloc[i,5])
        tmp.insert(0,0)
        new_probs.append(tmp)

In [9]:
print(len(new_label))
print(len(new_probs))

7765
7765


In [10]:
all_df = [new_label, new_probs]
all_df = pd.DataFrame(all_df).T
all_df = all_df.reset_index()
all_df

,index,0,1
0,0,org:product,"[0, 0.0028660793, 0.12103854, 0.85007775, 0.00..."
1,1,per:alternate_names,"[0, 0.0001525762, 0.00024262846, 0.00033883585..."
2,2,no_relation,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,3,no_relation,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
4,4,no_relation,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
...,...,...,...
7760,7760,no_relation,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
7761,7761,no_relation,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
7762,7762,org:top_members/employees,"[0, 0.99944526, 2.1029215e-05, 1.3179038e-05, ..."
7763,7763,per:date_of_birth,"[0, 0.0004302161, 0.00025689378, 0.000724003, ..."


In [11]:
all_df.columns = ['id','pred_label', 'probs']

In [12]:
all_df.to_csv('output_new1.csv')